В данном ноутбуке будет произведен семантический анализ описания вакансии и выделение ключевых навыков

In [1]:
import sys
import gensim, logging
from pymystem3 import Mystem
import wget
import zipfile
from collections import Counter
import pandas as pd

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
model_url = 'http://vectors.nlpl.eu/repository/11/180.zip'

Для загрузки модели надо запустить следующие 2 ячейки

In [4]:
#запускать 1 раз. Грузит модель
modelfile = wget.download(udpipe_url)

100% [........................................................................] 40616122 / 40616122

In [7]:
#запускать 1 раз. Грузит другую модель
m = wget.download(model_url)
model_file = model_url.split('/')[-1]

Создаем модель

In [ ]:
model_url = 'http://vectors.nlpl.eu/repository/11/180.zip'
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

Обрабатываем строки и получаем текст в нужном моделе формате

In [ ]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn

In [ ]:
import requests
import re

url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mapping = {}
r = requests.get(url, stream=True)
for pair in r.text.split('\n'):
    pair = re.sub('\s+', ' ', pair, flags=re.U).split(' ')
    if len(pair) > 1:
        mapping[pair[0]] = pair[1]

print(mapping)

In [ ]:
from ufal.udpipe import Model, Pipeline
import os
import re

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    print('\nLoading the model...', file=sys.stderr)
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    print('Processing input...', file=sys.stderr)
    for line in text:
        line = unify_sym(line.strip())
        output = process(process_pipeline, text=line)
        print(' '.join(output))

In [ ]:
def tag_mystem(text='Текст нужно передать функции в виде строки!'):  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if pos in mapping:
                tagged.append(lemma + '_' + mapping[pos]) # здесь мы конвертируем тэги
            else:
                tagged.append(lemma + '_X') # на случай, если попадется тэг, которого нет в маппинге
        except KeyError:
            continue # пропускаем знаки препинания
        except IndexError:
            continue
    return tagged

Пишем функцию для анализа всего текста

In [ ]:
def find_skills(skill_list, description_row):
    finale_skills = []
    correlation_rate = 0.4
    description = tag_mystem(text=description_row)
    for word in description:
        if not word in model:
            description.remove(word)
    for skill in skill_list:
        for word in description:
            if (word in model) and (model.similarity(skill, word) > correlation_rate):
                finale_skills.append((skill, word))
    return finale_skills

Загружаем данные и переводим навыки в хороший вид. В дальнейшем их можно будет также прогнать через алгоритм, чтобы понять к какому именно кластеру они относятся

In [ ]:
df = pd.read_csv('data.csv', sep=';', index_col=0)

In [ ]:
df['skill'] = df['key_skills'].apply(tag_mystem)

In [ ]:
skills = 'общительность трудолюбие аккуратность опрятность стрессоустойчивость пунктуальность ответственность выносливость исполнительность креативность творческий инженерный внимательность коммуникабельность точность программирование сварка грамотность английский этика фармацевтика дружелюбность сообразительность результативность уравновешенность пк исполнительность самостоятельность инициативность' 

In [ ]:
good_skills = tag_mystem(text = skills)

Удаляем все слова, которых нет в модели

In [18]:
for i in good_skills:
    if not i in model:
        good_skills.remove(i)

Все, в целом навыки собраны, осталось только просутить их через алгоритм по аналогии с описанием (см. дальше)

Переходим к анализу описания. Чтобы избежать случайных совпадений, мы будем брать только те навыки, которые употребляются часто. Следующие 2 ячейки работают очень долго, но результаты есть внизу

In [19]:
def finale_skills(description_row):
    sk0 = find_skills(good_skills, str(description_row))
    sk1 = list(map(lambda x: x[0], sk0))
    nums = Counter(sk1)
    sum_val = 0
    res = []
    array = list(nums.keys())
    for skill in array:
        sum_val += nums[skill]
    div = len(array)
    if len(array) == 0:
        div = 1
    avarage = sum_val / div
    for skill in array:
        if nums[skill] >= avarage:
            res.append(skill)
    return res

In [20]:
df['skills_from_descriprion'] = df['description'].apply(finale_skills)

In [21]:
for skill in good_skills:
    df[skill] = df['skills_from_descriprion'].apply(lambda x: int(skill in x))

Теперь посмотрим на частоту распространения навыков. Если какие-то будут слишком распространены, или же вовсе не встретятся, то мы их удалим.

In [35]:
(df[good_skills].sum()/df.shape[0])

трудолюбие_NOUN            0.1875
аккуратность_NOUN          0.1000
опрятность_NOUN            0.0250
пунктуальность_NOUN        0.0750
ответственность_NOUN       0.4000
выносливость_NOUN          0.0500
исполнительность_NOUN      0.4750
креативность_NOUN          0.0125
творческий_ADJ             0.4625
инженерный_ADJ             0.3000
внимательность_NOUN        0.1375
коммуникабельность_NOUN    0.1625
точность_NOUN              0.0250
программирование_NOUN      0.3875
сварка_NOUN                0.2875
электрик_NOUN              0.0000
грамотность_NOUN           0.1875
английский_ADJ             0.0000
этика_NOUN                 0.1875
фармацевтика_NOUN          0.1250
образование_NOUN           0.7250
сообразительность_NOUN     0.1375
результативность_NOUN      0.1500
серьезность_NOUN           0.0000
уравновешенность_NOUN      0.0625
пк_NOUN                    0.0000
исполнительность_NOUN      0.4750
самостоятельность_NOUN     0.1250
инициативность_NOUN        0.1875
dtype: float64

Теперь сгруппируем все признаки по 8 основным группам

In [67]:
#тут ручками объединяем группы, которые не видит алгоритм. В реальности, если мы хорошо натренируем собственную модель, то это будет не нужно
creativity= 'креативность_NOUN творческий_ADJ'.split()
hard_work = 'выносливость_NOUN трудолюбие_NOUN результативность_NOUN исполнительность_NOUN'.split()
accuracy = 'точность_NOUN внимательность_NOUN пунктуальность_NOUN аккуратность_NOUN'.split()
leader = 'инициативность_NOUN самостоятельность_NOUN ответственность_NOUN'.split()
computer = 'пк_NOUN программирование_NOUN'.split()
mind_tech = 'сообразительность_NOUN инженерный_ADJ'.split()
communication = 'общмительность_NOUN этика_NOUN коммуникабельность_NOUN опрятность_NOUN уравновешенность_NOUN'.split()
language = 'английский_ADJ грамотность_NOUN'.split()

Теперь сожмем навыки из таблицы в эти 8 категорий

In [79]:
finale_skills = []

def compress(name, columns):
    df[name] =  0
    for col in columns:
        df[name] += df[col]
    df[name + '_num'] = (df[name] > 0).apply(lambda x: int(x))
    finale_skills.append(name + '_num')
    return df.drop(name, axis=1)

In [80]:
df = compress('creativity', creativity)
df = compress('hard_work', hard_work)
df = compress('accuracy', accuracy)
df = compress('leader', leader)
df = compress('computer', computer)
df = compress('mind_tech', mind_tech)
df = compress('communication', communication)
df = compress('language', language)

Выводим таблицу с полученными навыками

In [84]:
df[finale_skills]

,creativity_num,hard_work_num,accuracy_num,leader_num,computer_num,mind_tech_num,communication_num,language_num
0,1,1,0,1,1,0,0,0
1,1,1,1,1,1,1,1,1
2,0,0,0,0,1,0,0,0
3,1,1,0,1,0,0,1,1
4,1,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...
75,0,1,1,1,1,1,1,0
76,0,0,0,0,0,0,0,0
77,1,1,1,1,0,1,0,1
78,0,1,0,1,1,0,1,1


Дальше ее можно сохранить и обучать на ней модель